In [ ]:
import sys
sys.path.append('..')
from ipywidgets import Textarea, VBox, Button, Output, IntProgress, Layout, Dropdown
from IPython.display import display, clear_output, HTML
import time
import traceback
from src.core import process_order_request, process_ride_plan_request, process_turtle_request

# --- Define User Queries for All Three Domains ---
tax_queries = [
    "We had two burgers at 10 euros each and a large soda for 3 euros.",
    "I want the fancy steak for 150 and a glass of vintage wine for 80.",
    "My order was one salad for 60 euro."
]
cycling_queries = [
    "I want to plan a long, flat ride. Let's say 100 kilometers.",
    "Plan an epic 500km ride across the desert.",
    "Let's plan a 75km ride through some jungle terrain."
]
turtle_queries = [
    "Draw a blue square with sides of length 100.",
    "Can you draw a red star?"
]
print("✅ System ready. Three domains are configured.")

In [ ]:
def create_multi_domain_ui():
    domain_selector = Dropdown(
        options=['Tax Calculation', 'Cycling Ride Planner', 'Turtle Graphics'],
        value='Tax Calculation',
        description='Select Domain:',
        style={'description_width': 'initial'}
    )
    query_input = Textarea(
        value=tax_queries[0],
        description='User Query:',
        style={'description_width': 'initial'},
        layout={'width': '95%', 'height': '100px'}
    )
    run_button = Button(description='Generate DSL & Execute', button_style='success', icon='play')
    progress_bar = IntProgress(value=0, min=0, max=10, description='Waiting...', bar_style='info', orientation='horizontal', layout={'visibility': 'hidden'})
    output_area = Output()

    def on_domain_change(change):
        if change['new'] == 'Tax Calculation':
            query_input.value = tax_queries[0]
        elif change['new'] == 'Cycling Ride Planner':
            query_input.value = cycling_queries[0]
        else: # Turtle Graphics
            query_input.value = turtle_queries[0]
    domain_selector.observe(on_domain_change, names='value')

    def run_analysis(button):
        with output_area:
            clear_output(wait=True)
            run_button.disabled = True
            progress_bar.description = 'LLM processing...'; progress_bar.layout.visibility = 'visible'; progress_bar.value = 5

            domain = domain_selector.value
            query = query_input.value
            
            if domain == 'Tax Calculation':
                result = process_order_request(query)
            elif domain == 'Cycling Ride Planner':
                result = process_ride_plan_request(query)
            else: # Turtle Graphics
                result = process_turtle_request(query)

            progress_bar.value = 10
            if result["status"] == "success":
                display(HTML("<h4>1. LLM-Generated DSL Code</h4>"))
                display(HTML(f"<pre style='background-color:#f5f5f5; border: 1px solid #ccc; padding: 10px; border-radius: 5px;'>{result['llm_generated_dsl']}</pre>"))

                display(HTML("<h4>2. Interpreter Execution Result</h4>"))
                dsl_result = result['interpreter_result']

                if domain == 'Tax Calculation':
                    display(HTML(f"""
                    <table style='width:100%; border-collapse: collapse;'>
                      <tr style='background-color:#f2f2f2;'><th style='text-align:left; padding:8px; border: 1px solid #ddd;'>Component</th><th style='text-align:right; padding:8px; border: 1px solid #ddd;'>Calculated Value</th></tr>
                      <tr><td style='padding:8px; border: 1px solid #ddd;'>Net Food Cost</td><td style='text-align:right; padding:8px; border: 1px solid #ddd;'>€{dsl_result['net_food_cost']:.2f}</td></tr>
                      <tr><td style='padding:8px; border: 1px solid #ddd;'>Net Drink Cost</td><td style='text-align:right; padding:8px; border: 1px solid #ddd;'>€{dsl_result['net_drink_cost']:.2f}</td></tr>
                      <tr><td style='padding:8px; border: 1px solid #ddd;'>Total Tax</td><td style='text-align:right; padding:8px; border: 1px solid #ddd;'>€{dsl_result['total_tax']:.2f}</td></tr>
                      <tr style='font-weight:bold; background-color:#f2f2f2;'><td style='padding:8px; border: 1px solid #ddd;'>FINAL BILL</td><td style='text-align:right; padding:8px; border: 1px solid #ddd;'>€{dsl_result['final_bill']:.2f}</td></tr>
                    </table>"""))
                elif domain == 'Cycling Ride Planner':
                    display(HTML(f"""
                    <p>The interpreter calculated the following based on the DSL:</p>
                    <ul>
                        <li><b>Distance:</b> {dsl_result['input_properties'].get('distance_km', 0)} km</li>
                        <li><b>Terrain:</b> {dsl_result['input_properties'].get('terrain', 'N/A')}</li>
                        <li style='font-weight:bold;'>Estimated Duration: {dsl_result['estimated_duration_hours']:.2f} hours</li>
                    </ul>
                    """))
                else: # Turtle Graphics
                    display(HTML(f"<p style='color:green; font-weight:bold;'>{dsl_result['message']}</p>"))
            else:
                display(HTML(f"<h4 style='color:red;'>🔴 PROCESSING ERROR</h4><p>{result['message']}</p>"))

            run_button.disabled = False; progress_bar.layout.visibility = 'hidden'; progress_bar.value = 0
    run_button.on_click(run_analysis)
    print("--- Multi-Domain DSL Validator ---")
    display(VBox([domain_selector, query_input, run_button, progress_bar, output_area]))

create_multi_domain_ui()